# Visualize pymoo result
Load the pymoo result object saved with pickle and visualize it.

ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

### Load nsga2 results for all topos

In [2]:
with open('res_bc_Cogent_nsga2.pkl','rb') as f_Cogent:
    res_3_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_nsga2.pkl','rb') as f_UsCarrier:
    res_3_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_nsga2.pkl','rb') as f_HiberniaGlobal:
    res_3_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_nsga2.pkl','rb') as f_Colt:
    res_3_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_nsga2.pkl','rb') as f_Funet:
    res_3_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_nsga2.pkl','rb') as f_Abvt:
    res_3_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_nsga2.pkl','rb') as f_Intellifiber:
    res_3_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_nsga2.pkl','rb') as f_TataNld:
    res_3_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_nsga2.pkl','rb') as f_Internode:
    res_3_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_nsga2.pkl','rb') as f_Missouri:
    res_3_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_nsga2.pkl','rb') as f_Ion:
    res_3_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_nsga2.pkl','rb') as f_Palmetto:
    res_3_Palmetto = pickle.load(f_Palmetto)

## Hypervolume
1. Store values

In [ ]:

# nsga2
F3_Cogent=res_3_Cogent.F
F3_UsCarrier=res_3_UsCarrier.F
F3_HiberniaGlobal=res_3_HiberniaGlobal.F
F3_Colt=res_3_Colt.F
F3_Funet=res_3_Funet.F
F3_Abvt=res_3_Abvt.F
F3_Intellifiber=res_3_Intellifiber.F
F3_TataNld=res_3_TataNld.F
F3_Internode=res_3_Internode.F
F3_Missouri=res_3_Missouri.F
F3_Ion=res_3_Ion.F
F3_Palmetto=res_3_Palmetto.F

# # Nadir Point from 9 Algorithms: 
# ref_point = [1.81000000e+02 6.20000000e+01 3.92281875e+03 1.70537209e+00]
# # Nadir Point from 4 Algorithms: 
# ref_point = [160.  77.  inf  inf]
# # Nadir Point from all (13) Algorithms: 
# ref_point = [181.  77.  inf  inf]

ref_point = [181, 77, 3922.81875, 1.70537209]

hist_F3_Cogent = []
hist_F3_UsCarrier = []
hist_F3_HiberniaGlobal = []
hist_F3_Colt = []
hist_F3_Funet = []
hist_F3_Abvt = []
hist_F3_Intellifiber = []
hist_F3_TataNld = []
hist_F3_Internode = []
hist_F3_Missouri = []
hist_F3_Ion = []
hist_F3_Palmetto = []


for algo3_Cogent in res_3_Cogent.history:
    opt3_Cogent = algo3_Cogent.opt
    feas3_Cogent = np.where(opt3_Cogent.get("feasible"))[0]
    hist_F3_Cogent.append(opt3_Cogent.get("F")[feas3_Cogent])
for algo3_UsCarrier in res_3_UsCarrier.history:
    opt3_UsCarrier = algo3_UsCarrier.opt
    feas3_UsCarrier = np.where(opt3_UsCarrier.get("feasible"))[0]
    hist_F3_UsCarrier.append(opt3_UsCarrier.get("F")[feas3_UsCarrier])
for algo3_HiberniaGlobal in res_3_HiberniaGlobal.history:
    opt3_HiberniaGlobal = algo3_HiberniaGlobal.opt
    feas3_HiberniaGlobal = np.where(opt3_HiberniaGlobal.get("feasible"))[0]
    hist_F3_HiberniaGlobal.append(opt3_HiberniaGlobal.get("F")[feas3_HiberniaGlobal])
for algo3_Colt in res_3_Colt.history:
    opt3_Colt = algo3_Colt.opt
    feas3_Colt = np.where(opt3_Colt.get("feasible"))[0]
    hist_F3_Colt.append(opt3_Colt.get("F")[feas3_Colt])
for algo3_Funet in res_3_Funet.history:
    opt3_Funet = algo3_Funet.opt
    feas3_Funet = np.where(opt3_Funet.get("feasible"))[0]
    hist_F3_Funet.append(opt3_Funet.get("F")[feas3_Funet])
for algo3_Abvt in res_3_Abvt.history:
    opt3_Abvt = algo3_Abvt.opt
    feas3_Abvt = np.where(opt3_Abvt.get("feasible"))[0]
    hist_F3_Abvt.append(opt3_Abvt.get("F")[feas3_Abvt])
for algo3_Intellifiber in res_3_Intellifiber.history:
    opt3_Intellifiber = algo3_Intellifiber.opt
    feas3_Intellifiber = np.where(opt3_Intellifiber.get("feasible"))[0]
    hist_F3_Intellifiber.append(opt3_Intellifiber.get("F")[feas3_Intellifiber])
for algo3_TataNld in res_3_TataNld.history:
    opt3_TataNld = algo3_TataNld.opt
    feas3_TataNld = np.where(opt3_TataNld.get("feasible"))[0]
    hist_F3_TataNld.append(opt3_TataNld.get("F")[feas3_TataNld])
for algo3_Internode in res_3_Internode.history:
    opt3_Internode = algo3_Internode.opt
    feas3_Internode= np.where(opt3_Internode.get("feasible"))[0]
    hist_F3_Internode.append(opt3_Internode.get("F")[feas3_Internode])
for algo3_Missouri in res_3_Missouri.history:
    opt3_Missouri = algo3_Missouri.opt
    feas3_Missouri = np.where(opt3_Missouri.get("feasible"))[0]
    hist_F3_Missouri.append(opt3_Missouri.get("F")[feas3_Missouri])
for algo3_Ion in res_3_Ion.history:
    opt3_Ion = algo3_Ion.opt
    feas3_Ion = np.where(opt3_Ion.get("feasible"))[0]
    hist_F3_Ion.append(opt3_Ion.get("F")[feas3_Ion])
for algo3_Palmetto in res_3_Palmetto.history:
    opt3_Palmetto = algo3_Palmetto.opt
    feas3_Palmetto = np.where(opt3_Palmetto.get("feasible"))[0]
    hist_F3_Palmetto.append(opt3_Palmetto.get("F")[feas3_Palmetto])

2. Calculate hypervolume and merge them (Mean value) for nsga2 algorithm

In [ ]:
from pymoo.indicators.hv import Hypervolume

metric = Hypervolume(ref_point= np.array(ref_point))

hv_3_Cogent = [metric.do(_F3_Cogent) for _F3_Cogent in hist_F3_Cogent]
hv_3_UsCarrier = [metric.do(_F3_UsCarrier) for _F3_UsCarrier in hist_F3_UsCarrier]
hv_3_HiberniaGlobal = [metric.do(_F3_HiberniaGlobal) for _F3_HiberniaGlobal in hist_F3_HiberniaGlobal]
hv_3_Colt = [metric.do(_F3_Colt) for _F3_Colt in hist_F3_Colt]
hv_3_Funet = [metric.do(_F3_Funet) for _F3_Funet in hist_F3_Funet]
hv_3_Abvt = [metric.do(_F3_Abvt) for _F3_Abvt in hist_F3_Abvt]
hv_3_Intellifiber = [metric.do(_F3_Intellifiber) for _F3_Intellifiber in hist_F3_Intellifiber]
hv_3_TataNld = [metric.do(_F3_TataNld) for _F3_TataNld in hist_F3_TataNld]
hv_3_Internode = [metric.do(_F3_Internode) for _F3_Internode in hist_F3_Internode]
hv_3_Missouri = [metric.do(_F3_Missouri) for _F3_Missouri in hist_F3_Missouri]
hv_3_Ion = [metric.do(_F3_Ion) for _F3_Ion in hist_F3_Ion]
hv_3_Palmetto = [metric.do(_F3_Palmetto) for _F3_Palmetto in hist_F3_Palmetto]

# nsga2
hv_3 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_3_Cogent, hv_3_UsCarrier, hv_3_HiberniaGlobal, hv_3_Colt, hv_3_Funet, hv_3_Abvt, hv_3_Intellifiber, hv_3_TataNld, hv_3_Internode, hv_3_Missouri, hv_3_Ion, hv_3_Palmetto)]

In [ ]:
print('The hypervolumn for Cogent:')
print(hv_3_Cogent)
print('The hypervolumn for UsCarrier:')
print(hv_3_UsCarrier)
print('The hypervolumn for HiberniaGlobal:')
print(hv_3_HiberniaGlobal)
print('The hypervolumn for Colt:')
print(hv_3_Colt)
print('The hypervolumn for Funet:')
print(hv_3_Funet)
print('The hypervolumn for Abvt:')
print(hv_3_Abvt)
print('The hypervolumn for Intellifiber:')
print(hv_3_Intellifiber)
print('The hypervolumn for TataNld:')
print(hv_3_TataNld)
print('The hypervolumn for Internode:')
print(hv_3_Internode)
print('The hypervolumn for Missouri:')
print(hv_3_Missouri)
print('The hypervolumn for Ion:')
print(hv_3_Ion)
print('The hypervolumn for Palmetto:')
print(hv_3_Palmetto)

# Show all hv for each topo
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_3_Cogent)+1)), hv_3_Cogent,  color='black', label='Cogent')
plt.plot(list(range(1, len(hv_3_UsCarrier)+1)), hv_3_UsCarrier,  color='blue', label='UsCarrier')
plt.plot(list(range(1, len(hv_3_HiberniaGlobal)+1)), hv_3_HiberniaGlobal,  color='red', label='HiberniaGlobal')
plt.plot(list(range(1, len(hv_3_Colt)+1)), hv_3_Colt,  color='orange', label='Colt')
plt.plot(list(range(1, len(hv_3_Funet)+1)), hv_3_Funet,  color=(0, 0, 1, 0.5), label='Funet')
plt.plot(list(range(1, len(hv_3_Abvt)+1)), hv_3_Abvt,  color='gray', label='Abvt')
plt.plot(list(range(1, len(hv_3_Intellifiber)+1)), hv_3_Intellifiber,  color='purple', label='Intellifiber')
plt.plot(list(range(1, len(hv_3_TataNld)+1)), hv_3_TataNld,  color='green', label='TataNld')
plt.plot(list(range(1, len(hv_3_Internode)+1)), hv_3_Internode,  color='#33FF57', label='Internode')
plt.plot(list(range(1, len(hv_3_Missouri)+1)), hv_3_Missouri,  color='brown', label='Missouri')
plt.plot(list(range(1, len(hv_3_Ion)+1)), hv_3_Ion,  color='pink', label='Ion')
plt.plot(list(range(1, len(hv_3_Palmetto)+1)), hv_3_Palmetto,  color='olive', label='Palmetto')
plt.title("Convergence")
plt.xlabel("Topos")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

In [ ]:
print('The aveage of hypervolumn for nsga2:')
print(hv_3)

3. Draw mean hypervolume for nsga2

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations") # brown,pink,#5733FF,olive,cyan,#FF5733,#33FF57
plt.ylabel("Hypervolume")
plt.legend()
plt.show()